In [1]:
from utils import *
import json
import logging
import matplotlib.pyplot as plt

In [2]:
driver = webdriver.Firefox()

# Initialise Google.
driver.get("https://www.google.com/maps")
driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Reject all"]').click()


In [ ]:
with open("data/codes.json", "r") as file:
    codes = json.load(file)

In [36]:
with open("data/stations.json", "r") as file:
    stations, lines, areas = json.load(file).values()

In [4]:
with open("data/nodes.json", "r") as file:
    nodes = json.load(file)

In [5]:
with open("data/times.npy", "rb") as file:
    mat = np.load(file)

In [6]:
with open("data/url.json", "r") as file:
    url_dict = json.load(file)

In [7]:
driver.get("https://en.wikipedia.org/wiki/List_of_London_railway_stations")

In [ ]:
stations_all: list[str] = [station.text for station in driver.find_elements(By.CSS_SELECTOR, ".wikitable:first-of-type tbody tr td:first-child > a")]
areas_all: list[str] = [area.text for area in driver.find_elements(By.CSS_SELECTOR, ".wikitable tbody tr td:last-child > a")]

In [28]:
codes: list[str] = [code.text for code in driver.find_elements(By.CSS_SELECTOR, ".wikitable tbody tr td:nth-child(4)")]

In [39]:
i = 0
for station in stations:
    if station in stations_all:
        i += 1
print(i)

59


In [ ]:

unique_stations, counts = np.unique(stations, 
                                    return_counts=True)
for station in unique_stations[counts > 1]:
    i = stations.index(station)
    stations.pop(i+1)
    lines[i] = lines[i] + lines[i+1]
    areas.pop(i+1)

{
    "stations": stations,
    "lines": lines,
    "areas": areas
}

In [ ]:
# Initialise mat.
# n = len(stations)
# mat = np.zeros((n, n), np.int8) 

In [150]:
line = "northern"
route = "fyc_mhl"
url = "https://www.google.com/maps/dir/Finchley+Central+Station,+Station+Road,+London/Mill+Hill+East,+Bittacy+Hill,+London/@51.6047157,-0.2062923,16z/am=t/data=!4m14!4m13!1m5!1m1!1s0x48761754925a9199:0x607b114fc4a08177!2m2!1d-0.1925137!2d51.6010896!1m5!1m1!1s0x48761748679003d7:0x65ab5ad9118f519b!2m2!1d-0.2099414!2d51.6083227!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYxMS4wIKXMDSoASAFQAw%3D%3D"
if line in url_dict.keys():
    url_dict[line][route] = url
else:
    url_dict[line] = { route: url }


In [151]:
# Requires user input to access and open travel details.
driver.get(url_dict[line][route])

In [152]:
journey, timestamps = get_journey(driver).values()

Number of stations: 2


In [153]:
# Calculate travel time between stations.
times = []
for i in range(len(timestamps) - 1):
    time: int = max((timestamps[i+1] - timestamps[i]).seconds//60, 1)
    times.append(time)

In [154]:
journey

['Finchley Central', 'Mill Hill East']

In [155]:
beta: float = 0.9

# Update time matrix.
for i in range(len(times)):
    start_ind = stations.index(journey[i])
    end_ind = stations.index(journey[i+1])
    prev_time = mat[start_ind, end_ind]
    if prev_time == 0:
        mat[start_ind, end_ind] = times[i]
    else:
        mat[start_ind, end_ind] = (1 - beta) * times[i] + beta * prev_time
    mat[end_ind, start_ind] = mat[start_ind, end_ind]
    print(f"{i}. Updated ({start_ind}, {end_ind}) from {prev_time} to {mat[start_ind, end_ind]}.")

0. Updated (79, 146) from 0.0 to 1.0.


In [156]:
if line in nodes.keys():
    nodes[line][route] = journey
else:
    nodes[line] = { route: journey }

In [157]:
with open("data/times.npy", "wb") as file:
    np.save(file, mat)

In [158]:
with open("data/nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)

In [159]:
with open("data/url.json", "w") as file:
    data = json.dumps(url_dict, indent=4)
    file.write(data)

In [81]:
driver.quit()

In [ ]:
start = "South Kensington"
travel_time: float = 45
stations_ind = []
start_ind: int = stations.index(start)
find_stations_in_range(mat, start_ind, travel_time, stations_ind)


In [ ]:
set(areas[i] for i in stations_ind)

In [ ]:
# node_pos = np.zeros(len(stations))

In [10]:
piccadilly_nodes = np.zeros((len(stations), 3))

In [53]:
piccadilly_nodes[stations.index("Heathrow Terminal 5")]

array([14.,  9.,  2.])

In [26]:
for i in range(len(nodes["piccadilly"]["act_hiv"])):
    piccadilly_nodes[stations.index(nodes["piccadilly"]["act_hiv"][i])] = [14,i,0]

In [15]:
circle_nodes[stations.index("Bayswater")]

array([25.,  0.])

In [16]:
with open("circle.npy", "wb") as file:
    np.save(file, circle_nodes)

In [69]:
nodes = np.hstack((nodes, np.zeros((269,2))))

In [ ]:
nodes = np.zeros((len(stations), 2))

In [90]:
nodes[stations.index("Hammersmith")]

array([7., 0., 0., 0.])

In [91]:
nodes[stations.index("Edgware Road")]

array([10.,  0.,  5.,  0.])

In [92]:
nodes[stations.index("Paddington")]

array([10.,  0.,  4.,  0.])

In [83]:
i = 0
for station in journey:
    print(station)
    nodes[stations.index(station)] = [10,0,i,0]
    i -= 1

Earl's Court
West Brompton
Fulham Broadway
Parsons Green
Putney Bridge
East Putney
Southfields
Wimbledon Park
Wimbledon


In [51]:
with open("piccadilly.npy", "wb") as file:
    np.save(file, piccadilly_nodes)

In [ ]:
driver.quit()

In [28]:
with open("nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)